In [20]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_audience_overlap_nodes, export_model_as_feature
from train import run_experiment

# Load audience overlap edges for level 1

In [21]:
level = 1
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2018_audience_overlap_sites_scrapping_result.json'), level=level)
audience_overlap_sites_NODES = create_audience_overlap_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-12 11:37:54 notebooks.utils INFO     Loaded 4238 nodes with records level <= 1 and child size:20335


[('villagevoice.com', 'nylon.com'), ('villagevoice.com', 'slantmagazine.com'), ('villagevoice.com', 'gothamist.com'), ('villagevoice.com', 'screendaily.com'), ('villagevoice.com', 'amny.com')]


In [22]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,villagevoice.com,nylon.com
1,villagevoice.com,slantmagazine.com
2,villagevoice.com,gothamist.com
3,villagevoice.com,screendaily.com
4,villagevoice.com,amny.com


### Find all unique nodes in edges

In [23]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 11865 Sample: ['yomiuri.co.jp', 'napims.com', 'nbcdfw.com', 'bradford-delong.com', 'hybridcenter.org']


### 1. Load all node features

In [24]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [25]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [26]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, yomiuri.co.jp to solvangusa.com
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   8657 non-null   float64
 1   daily_pageviews_per_visitor  8658 non-null   float64
 2   daily_time_on_site           6458 non-null   float64
 3   total_sites_linking_in       11469 non-null  float64
 4   bounce_rate                  6022 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [27]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11865 entries, yomiuri.co.jp to solvangusa.com
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   11865 non-null  float64
 1   daily_pageviews_per_visitor  8658 non-null   float64
 2   daily_time_on_site           6458 non-null   float64
 3   total_sites_linking_in       11865 non-null  float64
 4   bounce_rate                  6022 non-null   float64
dtypes: float64(5)
memory usage: 556.2+ KB


### 3. Normalizing features

In [28]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [29]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 11865, Edges: 20399

 Node types:
  default: [11865]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [20399]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [30]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [31]:
history = model.fit(train_gen, epochs=epochs, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/4
1899/1899 - 20s - loss: 0.6940 - binary_accuracy: 0.5283 - 20s/epoch - 10ms/step
Epoch 2/4
1899/1899 - 19s - loss: 0.6680 - binary_accuracy: 0.5432 - 19s/epoch - 10ms/step
Epoch 3/4
1899/1899 - 19s - loss: 0.6608 - binary_accuracy: 0.5427 - 19s/epoch - 10ms/step
Epoch 4/4
1899/1899 - 19s - loss: 0.6588 - binary_accuracy: 0.5419 - 19s/epoch - 10ms/step


In [32]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

# print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

238/238 [==============================] - 0s 735us/step


KeyError: 'crooked.com'

# Export embeddings as feature

In [33]:
export_model_as_feature(embeddings_wv, f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}', data_year='2018')

'/Users/panayot/Documents/News-Media-Peers/data/emnlp2018/features/attrib2vec_audience_overlap_level_1_epochs_4.json'

In [34]:
run_experiment(features=f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}', dataset='emnlp2018')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1_epochs_4 |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 11:40:35 train        INFO     Start training...
01-12 11:40:35 train        INFO     Fold: 0
01-12 11:40:42 train        INFO     Fold: 1
01-12 11:40:45 train        INFO     Fold: 2
01-12 11:40:49 train        INFO     Fold: 3
01-12 11:40:54 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+--------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1     |      Accuracy      |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+--------------------+--------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1_epochs_4 | 41.99783990229633 | 57.869934024505184 | 15.551366635249764 | 0.5768143261074458 |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+--------------------+--------------------+--------------------+


In [35]:
run_experiment(features=f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}', dataset='emnlp2018', task='bias')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| bias |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1_epochs_4 |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 11:41:23 train        INFO     Start training...
01-12 11:41:23 train        INFO     Fold: 0
01-12 11:41:29 train        INFO     Fold: 1
01-12 11:41:33 train        INFO     Fold: 2
01-12 11:41:37 train        INFO     Fold: 3
01-12 11:41:41 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1     |      Accuracy     |  Flip error-rate   |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+
| bias |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_1_epochs_4 | 36.64642540433258 | 55.32516493873704 | 5.5607917059377945 | 0.5023562676720076 |
+------+---------------------+---------------+--------------------+----------------------------------------------+-------------------+-------------------+--------------------+--------------------+
